In [1]:
using Pkg.TOML

using Random: seed!
using RMMMDNets

# enable_gpu()

;

In [8]:
rmmmdnets_path = pathof(RMMMDNets) |> splitdir |> first |> splitdir |> first
hyper = TOML.parsefile("$rmmmdnets_path/examples/Hyper.toml")

# dataset = "gaussian"
dataset = "ring"
# dataset = "mnist"

model_name = "gan"
# model_name = "mmdnet"
# model_name = "rmmmdnet"

args_dict = parse_toml(hyper, dataset, model_name)
args = parse_args_dict(args_dict; override=(D_z=10,))

;

┌ Info: Args
│   batch_size_gen = 200
│   batch_size = 200
│   act = relu
│   Dg_h = 100,100
│   D_z = 10
│   dataset = ring
│   act_last = identity
│   opt = adam
│   model_name = gan
│   n_epochs = 1000
│   Dd_h = 100,100
│   beta1 = 0.5
│   base = gaussian
│   lr = 0.0001
│   seed = 1
└ @ RMMMDNets /afs/inf.ed.ac.uk/user/s16/s1672897/projects/RMMMDNets.jl/src/RMMMDNets.jl:34


In [ ]:
function run_exp(args)
    data = get_data(args.dataset)
    
    seed!(args.seed)

    model = get_model(args, data)

    evaluate(data, model)

    dataloader = DataLoader(data, args.batch_size)
    
    train!(model, args.n_epochs, dataloader)
    
    return data, model
end

data, model = run_exp(args)

;